In [1]:
%%HTML
<style>
    div#notebook-container    { width: 100%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import collections
import datetime
import glob
import os
import re
import sqlite3
import time
import urllib

import lhafile
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

COLUMN_LIST = ['RACE_TIME', 'RACER_ID', 'PLACE', 'DISTANCE', 'COURSE', 'EXHIBITION_TIME', 'RACE_DATE']

class RaceResults:
    def __init__(self):
        self.baseuri = "http://www1.mbrace.or.jp/od2/K/%s/k%s.lzh" # http://www1.mbrace.or.jp/od2/K/201612/k161201.lzh
        self.results = [] # List of (RACE_TIME, RACER_ID, PLACE, COURSE, EXHIBITION_TIME)

    def download(self, start, end):
        period = pd.date_range(start, end)

        for date in period:
            # Get file from the website
            dirname = date.strftime("%Y%m")
            lzhname = date.strftime("%y%m%d")
            uri = self.baseuri % (dirname, lzhname)
            savename = "./data/results/lzh/%s.lzh" % lzhname
            if not os.path.exists(savename):
                print("Send request to", uri)
                urllib.request.urlretrieve(uri, savename)
                time.sleep(3)

            unpackedpath = "./data/results/K%s.TXT" % lzhname
            unpackedname = os.path.basename(unpackedpath)
            if not os.path.exists(unpackedpath):
                print("Unpacking", savename)
                f = lhafile.Lhafile(savename)
                data = f.read(unpackedname)
                datastr = data.decode(encoding='shift-jis')
                fileobj = open(unpackedpath, "w")
                fileobj.write(datastr)
                fileobj.close()

    def load_and_regist(self):
        all_results = []
        for filename in glob.glob("./data/results/K*.TXT"):
            race_date_str = filename.replace("./data/results/K", "20").replace(".TXT","")
            tdatetime = datetime.datetime.strptime(race_date_str, '%Y%m%d')
            race_date = datetime.date(tdatetime.year, tdatetime.month, tdatetime.day)
            with open(filename, "r") as f:
                get_place = False
                get_racer_info = False
                count = 1
                race_time = 0
                for line in f:
                    line_replaced =  line.replace("\u3000", "")
                    if line_replaced.find('BGN') > -1:
                        get_place = True
                    elif get_place:
                        _place_index = line_replaced.find('［成績］')
                        place = line_replaced[0:_place_index]
                        get_place = False
                    elif re.search(r'H\d{4}m', line_replaced, flags=0)  is not None:
                        distance = re.search(r'\d{4}', line_replaced, flags=0).group()
                    elif line_replaced.startswith("----"):
                        get_racer_info = True
                    elif get_racer_info and count < 7:
                        elems = line_replaced.split()
                        if elems[0] not in ['01','02','03','04','05','06']:
                            count += 1
                            continue
                        if elems[9] != '.':
                            _race_time = elems[9].split('.')
                            race_time = float(_race_time[0])*60 + float(_race_time[1]) + float(_race_time[2])/10
                        else:
                            race_time = race_time + 1
                        racer_id = elems[2]
                        course = float(elems[1])
                        exhibition_time = float(elems[6])
                        if race_date.month == 1:
                            all_results.append((race_time, racer_id, place, distance, course, exhibition_time, race_date))
                        all_results.append((race_time, racer_id, place, distance, course, exhibition_time, race_date))
                        count += 1
                    elif count == 7:
                        count = 1
                        get_racer_info = False
                        
        self.results = all_results
        regist(self.results)
                                                            
    def get_results_pd(self):
        return pd.DataFrame(get_results(), columns=COLUMN_LIST)

In [3]:
column_list_type = "RACE_TIME REAL, RACER_ID TEXT, PLACE TEXT, DISTANCE INTEGER, COURSE INTEGER, EXHIBITION_TIME REAL, RACE_DATE TEXT"
column_list = "RACE_TIME, RACER_ID, PLACE, DISTANCE, COURSE, EXHIBITION_TIME, RACE_DATE"
dbname = "raceresults.db"

def regist(results):
    conn=sqlite3.connect(dbname)
    cursor = conn.cursor()

    drop_table = "DROP TABLE IF EXISTS raceresults"
    cursor.execute(drop_table)
    create_table = "CREATE TABLE raceresults (" + column_list_type + ")"
    cursor.execute(create_table)

    sql = "INSERT INTO raceresults (" + column_list + ") Values (?,?,?,?,?,?,?)"
    for item in results:
        cursor.execute(sql, item)
    conn.commit()
    conn.close()

def get_results():
    conn=sqlite3.connect(dbname)
    cursor = conn.cursor()
    get_records = "SELECT * FROM raceresults"
    cursor.execute(get_records)
    results = cursor.fetchall()
    conn.close()
    return results

In [4]:
if __name__ == "__main__":
    r = RaceResults()
    r.download("2019-10-01","2020-01-31")
    r.load_and_regist()
    a = r.get_results_pd()

Send request to http://www1.mbrace.or.jp/od2/K/201910/k191001.lzh
Unpacking ./data/results/lzh/191001.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201910/k191002.lzh
Unpacking ./data/results/lzh/191002.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201910/k191003.lzh
Unpacking ./data/results/lzh/191003.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201910/k191004.lzh
Unpacking ./data/results/lzh/191004.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201910/k191005.lzh
Unpacking ./data/results/lzh/191005.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201910/k191006.lzh
Unpacking ./data/results/lzh/191006.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201910/k191007.lzh
Unpacking ./data/results/lzh/191007.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201910/k191008.lzh
Unpacking ./data/results/lzh/191008.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201910/k191009.lzh
Unpacking ./data/results/lzh/191009.lzh
Send request to http://www1.mbrace.or.jp/od2/K

Unpacking ./data/results/lzh/191217.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191218.lzh
Unpacking ./data/results/lzh/191218.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191219.lzh
Unpacking ./data/results/lzh/191219.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191220.lzh
Unpacking ./data/results/lzh/191220.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191221.lzh
Unpacking ./data/results/lzh/191221.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191222.lzh
Unpacking ./data/results/lzh/191222.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191223.lzh
Unpacking ./data/results/lzh/191223.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191224.lzh
Unpacking ./data/results/lzh/191224.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191225.lzh
Unpacking ./data/results/lzh/191225.lzh
Send request to http://www1.mbrace.or.jp/od2/K/201912/k191226.lzh
Unpacking ./data/results/lzh/191226.lzh
Send r

In [5]:
a

,RACE_TIME,RACER_ID,PLACE,DISTANCE,COURSE,EXHIBITION_TIME,RACE_DATE
0,106.3,4715,芦屋,1800,2,6.72,2020-01-21
1,106.3,4715,芦屋,1800,2,6.72,2020-01-21
2,107.7,4342,芦屋,1800,1,6.70,2020-01-21
3,107.7,4342,芦屋,1800,1,6.70,2020-01-21
4,109.4,5017,芦屋,1800,5,6.74,2020-01-21
5,109.4,5017,芦屋,1800,5,6.74,2020-01-21
6,109.6,3286,芦屋,1800,3,6.68,2020-01-21
7,109.6,3286,芦屋,1800,3,6.68,2020-01-21
8,110.6,4365,芦屋,1800,4,6.72,2020-01-21
9,110.6,4365,芦屋,1800,4,6.72,2020-01-21
